In [1]:
import sys
sys.path.append('../')
import common.helpers as declass

In [2]:
cables_connector = declass.connect_db('declassification_cables')

In [3]:
cables_tables = declass.show_tables(cables_connector)

In [4]:
# print_to_md(preview(cables_tables))
cables_tables

['classification_countries',
 'classification_doc',
 'classifications',
 'concept_doc',
 'concepts',
 'countries',
 'country_doc',
 'doc_counts',
 'docs',
 'from_to_sum',
 'network_docs',
 'network_nodes',
 'office_doc',
 'offices',
 'person_doc',
 'persons',
 'reference_doc',
 'tag_doc',
 'tag_doc_staging',
 'tagname_doc',
 'tagnames',
 'tags',
 'tags_staging',
 'tokens',
 'top_classifications',
 'top_countries',
 'top_network',
 'top_persons',
 'top_topics',
 'topic_doc',
 'topic_token',
 'topics',
 'urgency',
 'urgency_doc']

In [5]:
declass.df_sample(cables_connector, "concepts", 5)
# declass.df(cables_db, "concepts")

,id,name
0,1,VS
1,2,WHEAT
2,3,CORN
3,4,RICE
4,5,IMPORT QUOTAS
